In [50]:
import os 
from dotenv import load_dotenv
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# print(TAVILY_API_KEY)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
# print(GOOGLE_API_KEY)
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
# print(LANGCHAIN_API_KEY)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# print(GROQ_API_KEY)

In [51]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.3-70b-versatile")
print(llm.invoke("Hi").content)

It's nice to meet you. Is there something I can help you with or would you like to chat?


In [52]:
message=[{"role":"system","content":"You are a helpful assistant"},
         {"role":"user","content":"hi How are you doing?"}]

In [53]:
print(llm.invoke(message).content)

I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always happy to help and assist you with any questions or tasks you may have. How about you? How's your day going so far?


In [54]:
class Chatbot:
    def __init__(self,system=""):
        # defining role of system
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})

    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result

    def execute(self):
        llm = ChatGroq(model_name = "llama-3.3-70b-versatile" )
        result = llm.invoke(self.message)
        return result.content

In [55]:
bot = Chatbot(system="you are a helpful assistant")


In [56]:
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'}]

In [57]:
bot("Hi How are you?")

"I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always happy to help and assist with any questions or tasks you may have. How about you? How's your day going so far? Is there anything I can help you with?"

In [58]:
# bot.__call__("Tell me about yourself")
# bot.execute()
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'Hi How are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm always happy to help and assist with any questions or tasks you may have. How about you? How's your day going so far? Is there anything I can help you with?"}]

In [59]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use thought to describe your thoughts about the question you have been asked.
Use action to run one of the actions available to you - then return PAUSE
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate:4*7/3
Runs a calculator and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Return a summary from searching wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Exaple session:
Question: What is the capital of France?
Thought: I should look up France on wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get bsic conversation questions like "hi","hello","how are you?,
you have to answer "hi","hello","I am good".

""".strip()

### Pattern Breakdown
* ^ : This matches the start of the string. It means the string must begin with what follows.
* Action : This is a literal match. iT means the string must have the text "Action" at the beginning.
* (\w+) : The parantheses () define a capture group. This allows you to extract part of the string that matches this section. \w+ matches one or more characters (letters, digits and undersores). This will capture a word that follows "Action".
* : : this is a literal colon that seperates teh word natched by (\w+) from the rest of the string.
* (.*) : THis is another capture group, where .* matches any character (.) zero or more times (*), which emans it captures everything that comes after the second colon.

## What does it do?
This regex is looking for a string that:
* Starts with the text "Action".
* Has a word right after it, seperated by a colon.
* Then, after another colon, it captures everything that follow.

In [ ]:
# To parse the action part during execution
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [61]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php",params={
        "action":"query",
        "list" : "search",
        "srsearch":query,
        "format":"json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [74]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json",params={
        "sql": """
        select 
        blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body),0,1000) as text,
        blog_entry.created
        from 
        blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
        blog_entry_fts match escape_fts(:q)
        order by
        blog_entry_fts.rank
        limit
        1
        """.strip(),
        "_shape":"array",
        "q":query
    })
    return response.json()[0]["text"]

In [75]:
def calculate(number):
    return eval(number)

In [76]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate" : calculate,
    "simon_blog_search" : simon_blog_search
}

In [77]:
def query(question,max_turns=5):
    i=0
    bot = Chatbot(prompt)
    next_prompt = question
    while i<max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [66]:
query("hi")

hi


'hi'

In [67]:
query("How are you doing?")

I am good


'I am good'

In [69]:
print(query("fifteen * twenty-five"))

Thought: To find the product of fifteen and twenty-five, I can use the calculate action to multiply these numbers.

Action: calculate: 15*25
PAUSE
 -- running calculate 15*25
Observation: 375
Answer: The product of fifteen and twenty-five is 375.
Answer: The product of fifteen and twenty-five is 375.


In [80]:
query("Has Simon written about AI?")

Thought: To find out if Simon has written about AI, I should search his blog for the term "AI" or "Artificial Intelligence". This will give me a list of relevant articles or posts that Simon has written on the topic.

Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That s

'Answer: Yes, Simon has written about AI. He has published at least two posts on the topic, including "It\'s OK to call it Artificial Intelligence" and a follow-up post titled "What I should have said about the term Artificial Intelligence", where he discusses the importance of having high-quality conversations about AI and its integration into society.'

In [79]:
print("Thought: The observation shows that Simon has indeed written about AI, and has even published a follow-up post to clarify his thoughts on the topic. The post discusses the importance of having high-quality conversations about AI and its implications.\n\nAction: None needed, I have enough information to answer the question.\nAnswer: Yes, Simon has written about AI.'")

Thought: The observation shows that Simon has indeed written about AI, and has even published a follow-up post to clarify his thoughts on the topic. The post discusses the importance of having high-quality conversations about AI and its implications.

Action: None needed, I have enough information to answer the question.
Answer: Yes, Simon has written about AI.'


In [82]:
print(query("Tell me what simon blog has written about AI."))

Thought: To find out what Simon's blog has written about AI, I should search his blog for the term "AI". This will likely give me a list of articles or posts that mention AI, and I can use that information to provide an answer.

Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomple

In [83]:
print(query("Tell me about the capital of USA."))

Thought: To find the capital of the USA, I should look up the United States on wikipedia, as it will likely have information on the country's capital.

Action: wikipedia: United States
PAUSE
 -- running wikipedia United States
Observation: The <span class="searchmatch">United</span> <span class="searchmatch">States</span> of America (USA), also known as the <span class="searchmatch">United</span> <span class="searchmatch">States</span> (U.S.) or America, is a country located primarily in North America. It is a federal
Thought: The observation provided does not explicitly mention the capital of the USA. I should try a more specific search, focusing on the capital itself, to get a direct answer.

Action: wikipedia: Capital of the United States
PAUSE
 -- running wikipedia Capital of the United States
Observation: This is a list <span class="searchmatch">of</span> <span class="searchmatch">capital</span> cities <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span 